In [14]:
import spacy
import pandas as pd
from spacy import displacy
import os

In [15]:
# Cargar el archivo CSV
df = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\limpios\escritos_procesados.csv")

# Ver las primeras filas para entender la estructura
print(df.head())

                                           contenido        titulo
0  Clitofon, acusado por Sócrates de haber censur...      Clitofon
1  La correspondencia que sigue comprende trece c...        Cartas
2  ¿Qué es lo justo? Este es el comienzo , y el a...   De lo justo
3  Hé aquí otra serie de preguntas y respuestas d...  De la virtud
4  La vida no es más que un viaje; y es natural, ...        Axioco


In [16]:
# Carga modelo español large
nlp = spacy.load('es_core_news_lg')

In [17]:
# Ejemplo de un escrito
ejemplo_escrito = df["contenido"][2]  # Tercer escrito
doc_escrito = nlp(ejemplo_escrito)

In [18]:
# Visualización de dependencias
displacy.render(doc_escrito, style="dep", jupyter=True)

In [19]:
# Visualización de entidades
displacy.render(doc_escrito, style="ent", jupyter=True)

In [4]:
"""
Funcion procesamiento de texto
Tokenizacion
Lematizacion convertir palabras a su forma base
Eliminacion de stopwords (Quita palabras sin valor semántico)
"""
def procesar_escrito(texto):
    doc = nlp(texto)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

In [5]:
# Procesar el contenido de los escritos
df["contenido_procesado"] = df["contenido"].apply(procesar_escrito)

# Mostrar algunos ejemplos procesados
print(df[["titulo", "contenido_procesado"]].head())

         titulo                                contenido_procesado
0      Clitofon  clitofon acusado sócrates censurar conversació...
1        Cartas  correspondencia comprender trece carta dirigid...
2   De lo justo  justo comienzo asunto diálogo injusto ignoranc...
3  De la virtud  hé serie pregunta respuesta interés literario ...
4        Axioco  vida viaje natural necesario término término m...


In [6]:
def extraer_preguntas(texto):
    doc = nlp(texto)
    preguntas = [sent.text for sent in doc.sents if "?" in sent.text]
    return preguntas

In [7]:
df["preguntas_filosoficas"] = df["contenido"].apply(extraer_preguntas)

# Ejemplo de preguntas encontradas
for index, row in df.iterrows():
    print(f"{row['titulo']} - Preguntas Filosóficas:")
    for pregunta in row["preguntas_filosoficas"]:
        print(f"{pregunta}")

Clitofon - Preguntas Filosóficas:
Cartas - Preguntas Filosóficas:
De lo justo - Preguntas Filosóficas:
¿Qué es lo justo?
,¿Qué demonio ha inspirado a Simón el Socrático la idea de escribir diálogos para fastidio de los traductores, puesto que los hay, y de los lectores, suponiendo que los haya?, , , , , , ,
De la virtud - Preguntas Filosóficas:
Hé aquí otra serie de preguntas y respuestas de un interés literario y filosófico escaso, por no decir nulo, y que se pueden resumir en las tres preguntas y tres respuestas siguientes:,¿La virtud puede por su naturaleza ser enseñada?
,¿Será la virtud un don de la naturaleza, es decir, una cualidad natural?
,¿Pero si la virtud no procede ni de la enseñanza ni de la naturaleza, de donde viene? –De los dioses.
Axioco - Preguntas Filosóficas:
Morir es hacerse insensible, es decir, incapaz de sufrir; y entonces ¿cómo la muerte puede ser un mal?
¿Cuál es la edad que no tiene sus dolores?
¿Qué oficio, qué arte, qué función deja de dar lugar a disgustos

In [8]:
"""
Identificacion de conceptos Filosóficos Clave
Named Entity Recognition para identificar entidades relevantes (valores, conceptos, eventos)
"""
def extraer_entidades(texto):
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

In [9]:
df["conceptos_clave"] = df["contenido_procesado"].apply(extraer_entidades)

# Mostrar algunos conceptos extraídos
print(df[["titulo", "conceptos_clave"]].head())

         titulo                                    conceptos_clave
0      Clitofon  [(clitofon, PER), (sócrates, PER), (sócrates, ...
1        Cartas  [(dion dionisio platón, PER), (dion aristodoro...
2   De lo justo                                     [(simón, PER)]
3  De la virtud      [(hé serie, MISC), (aristides pericles, PER)]
4        Axioco                           [(sócrates axioco, PER)]


In [11]:
"""
Filtrado de Fragmentos Estilo Socrático
Detectamos fragmentos que siguen un patron de pregunta-respuesta o donde predomina el razonamientos dialéctico.
"""
def extraer_fragmentos_socraticos(texto):
    lineas = texto.split("\n")
    fragmentos = [linea for linea in lineas if "¿" in linea or ":" in linea]
    return " ".join(fragmentos)

In [12]:
df["fragmentos_socraticos"] = df["contenido"].apply(extraer_fragmentos_socraticos)

# Mostrar fragmentos seleccionados
print(df[["titulo", "fragmentos_socraticos"]].head())

         titulo                              fragmentos_socraticos
0      Clitofon                                                   
1        Cartas  La correspondencia que sigue comprende trece c...
2   De lo justo  ¿Qué es lo justo? Este es el comienzo , y el a...
3  De la virtud  Hé aquí otra serie de preguntas y respuestas d...
4        Axioco  La vida no es más que un viaje; y es natural, ...


In [13]:
# Definir el directorio de salida
output_directory = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed"

# Guardar los DataFrames procesados
df.to_csv(os.path.join(output_directory, "escritos_socraticos_procesados.csv"), index=False)
